# Find the best PER contract

PER (Plan d’Epargne Retraite) is a French investment account which allows you to prepare your retirement and reduce some tax. But there are so many banks and insurance company that offers various contract of PER.

The objective of this challenge is to find the `best PER contract of 2022` in the market.

The raw data is coming from https://www.francetransactions.com/per-plan-epargne-retraite-206/comparatif-per.html


## Data description

- Frais sur versements maximum : Des réductions importantes, jusqu'à l'annulation complète des frais sur versements (0%), peuvent être proposées aux épargnants, selon leur intermédiaire. (This tells you how much money you need to pay when you transfer money into your PER account, for example if it's 5%, and you put 100 euros into your account, you only invest 95 euros. With a 2% gain per year, it will take long time to just gain back your capital. **So, I highly recommend you to avoid all contracts that has more than 2% frais versement** vous conseillons donc d’éviter les contrats qui prévoient des frais de versement supérieurs à 2%.)


- Frais de gestion sur le fonds euros pour les supports assurance-vie, frais de gestion de compte pour les autres supports (compte-titres, etc.).

- Frais de gestion sur les unités de compte pour les supports en assurance-vie. Frais de gestion des produits sur les encours du contrat pour les autres supports (compte-titres, etc.).

- Taux publié par les assureurs, nets des frais de gestion, nets de prélèvements sociaux.

- Taux nets pour les épargnants, nets des prélèvements sociaux.

- Votes instantanés des lecteurs, 1 vote par contrat par lecteur (adresse IP) est autorisé. N'induit rien sur la qualité intrinsèque des contrats.

In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import os

In [2]:
local=True
if local:
    spark=SparkSession.builder.master("local[4]") \
                  .appName("PER_challenge")\
                  .getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("PER_challenge") \
                      .config("spark.kubernetes.container.image",os.environ['IMAGE_NAME']) \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "4") \
                      .config("spark.executor.memory","8g") \
                      .config('spark.jars.packages','com.crealytics:spark-excel_2.12:3.1.2_0.17.1') \
                      .getOrCreate()

22/08/09 12:48:41 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.184.146 instead (on interface ens33)
22/08/09 12:48:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/09 12:48:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
file_path="../../data/per.xls"

In [5]:
pdf=ps.read_excel(file_path, sheet_name='per', index_col=[0])

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


/home/pliu/.cache/pypoetry/virtualenvs/sparkcommonfunc-3iAQ1Rpl-py3.8/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1078: FutureWarning: convert_float is deprecated and will be removed in a future version
  return pd.read_excel(


In [7]:
pdf.head()

,Assureur/Support,Avis sur 5,Frais Vers.,Frais Gestion Fonds ?,Frais Gestion UC,Frais/rente,Fonds euros,Taux brut,Nombre SCPI,Nombre SCI,Nombre OPCI,Nombre ETF,Nombre UC
PER,,,,,,,,,,,,,
ABEILLE RETRAITE PLURIELLE,ABEILLE RETRAITE PROFESSIONNELLE,NaN,5.0,1.00,1.00,NaN,ABEILLE EURO PERP,NaN,0,0,0,0,80
AFER RETRAITE INDIVIDUELLE,ABEILLE,NaN,3.0,1.00,1.00,0.0,ABEILLE RP SECURITE RETRAITE,NaN,0,0,0,0,80
ALLIANZ PER HORIZON,ALLIANZ,NaN,4.8,0.85,0.85,NaN,ALLIANZ RETRAITE,NaN,0,0,0,0,92
AMBITION RETRAITE INDIVIDUELLE,LA MONDIALE,NaN,3.9,0.70,0.70,0.0,FONDS EUROS RETRAITE,NaN,0,0,0,0,0
AMPLI-PER LIBERTE,AMPLI-MUTUELLE,NaN,0.0,0.50,0.40,0.0,AMPLI PER EUROS,NaN,2,0,0,3,4


In [6]:
df = spark.createDataFrame(pdf)
df.show()

TypeError: Can not infer schema for type: <class 'str'>

In [17]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    elif f == 'float32': return FloatType()
    else: return StringType()

def define_structure(col_name:str, pandas_col_type):
    try: spark_col_type = equivalent_type(pandas_col_type)
    except: spark_col_type = StringType()
    return StructField(col_name, spark_col_type,nullable=True)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types):
        print(column)
        struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    print(p_schema)
    return spark.createDataFrame(pandas_df, p_schema)

In [18]:
df=pandas_to_spark(pdf)

Assureur/Support
Avis sur 5
Frais Vers.
Frais Gestion Fonds ?
Frais Gestion UC
Frais/rente
Fonds euros
Taux brut
Nombre SCPI
Nombre SCI
Nombre OPCI
Nombre ETF
Nombre UC
StructType(List(StructField(Assureur/Support,StringType,true),StructField(Avis sur 5,DoubleType,true),StructField(Frais Vers.,DoubleType,true),StructField(Frais Gestion Fonds ?,DoubleType,true),StructField(Frais Gestion UC,DoubleType,true),StructField(Frais/rente,DoubleType,true),StructField(Fonds euros,StringType,true),StructField(Taux brut,DoubleType,true),StructField(Nombre SCPI,LongType,true),StructField(Nombre SCI,LongType,true),StructField(Nombre OPCI,LongType,true),StructField(Nombre ETF,LongType,true),StructField(Nombre UC,LongType,true)))


TypeError: StructType can not accept object 'Assureur/Support' in type <class 'str'>

In [ ]:
df.show()